In [43]:
import json
import requests
from src import utils
import time
import os
import csv
import re

### Making Training set

In [46]:
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

In [50]:
data = utils.read_jsonlines(input_file)

In [71]:
len(data)

1319

In [66]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [67]:
def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
    prompt = f"질문: {kor_que}"
    output = f"풀이: {kor_ans} 정답: {ans_s}"
    return prompt, output

In [68]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [69]:
# from pprint import pprint

In [70]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [29]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s)
    
    
    input_lst.append(remove_line_slit(in_))
    output_lst.append(remove_line_slit(out_))

#     if cnt == 5:
#         break
#     else:
#         cnt += 1

In [32]:
len(data) - split_cnt

7310

In [30]:
len(input_lst)

7310

In [31]:
len(output_lst)

7310

In [33]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [34]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [35]:
output_file = output_path + '/kor-gsm8k-reason-train.csv'

In [36]:
output_file

'/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-reason-train.csv'

In [39]:
save_csv_file(output_file, input_lst, output_lst)

In [40]:
input_lst[1]

'질문: 웽은 베이비시팅을 시간당 $12로 받습니다. 어제 그녀는 50분 동안 베이비시팅을 했습니다. 그녀가 벌은 돈은 얼마인가요?'

In [41]:
output_lst[1]

'풀이: 웽은 1분당 12/60 = $0.2를 벌었습니다. 50분 동안 일한 결과, 그녀는 0.2 x 50 = $10을 벌었습니다. 정답: 10'

### Making Test Set

In [76]:
output_file = output_path + '/kor-gsm8k-reason-test.jsonl'

In [77]:
print(output_file)

/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-reason-test.jsonl


In [73]:
jsonl_writer = utils.JSONLWriter(output_file)

In [74]:
len(data)

1319

In [78]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s)
    
    ins['kor_question_reason'] = in_
    ins['kor_question_reason_answer'] = out_

    jsonl_writer.write_json_line(ins)

In [79]:
len(data) - split_cnt

1293

In [80]:
split_cnt

26